In [1]:
import rioxarray
import rasterio
import pandas as pd
import os
import geopandas
import ftplib
import re
import zipfile
from tqdm import tqdm

In [2]:
data_input_dir = "data/gpm-imerg"
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)

In [3]:
geodf.bounds

,minx,miny,maxx,maxy
0,18.406925,50.476079,19.680133,51.179343


In [4]:
gpm_imerg_zip_dir = "data/gpm-imerg/zip"
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
zips = os.listdir(gpm_imerg_zip_dir)
file_pattern = ".*.zip"
file_patter = re.compile(file_pattern)
zips = list(filter(file_patter.match, zips))
for zipf in zips:
    try:
        with zipfile.ZipFile(os.path.join(gpm_imerg_zip_dir, zipf), 'r') as zip_ref:
            zip_ref.extractall(gpm_imerg_extracted_dir)
    except:
        print("Error in: " + zipf)
        

Error in: 3B-MO-L.GIS.IMERG.20190601.V06B.zip


In [5]:
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
files = os.listdir(gpm_imerg_extracted_dir)
file_pattern = ".*V06B.tif"
file_patter = re.compile(file_pattern)
file_list = list(filter(file_patter.match, files))

In [6]:
df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(gpm_imerg_extracted_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data = data.rio.write_crs("EPSG:4326")
    masked = data.to_masked_array()
    datetime = f[18:22] + "-" + f[22:24] + "-" + f[24:26]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    #df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)

/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1977/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1977/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1977/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1977/1244022856.py:13: FutureWarning: The frame.append method 

In [7]:
df.to_pickle("data/gpm-imerg_df.pickle")